# Hidden Hits 1 - Data Collection
In this notebook, we focus on collecting data on songs that appeared in the Billboard Hot 100s Year End lists starting from the year 2010 to 2020. We focus on the collecting the following features:
- Title
- Artists
- Year
- Rank
- Lyrics

We build the dataframe by means of webscraping. For the first 4 features, we scrape the data from the Billboard Hot 100s website using **BeautifulSoup**. We use Genius to collect lyrics on all the songs we have scraped. Let's get started!

We first import all the relevant packages. We use `pandas` and `numpy` for data management, `BeautifulSoup` for webscraping and `os` and `re` for lyrics formatting

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from seaborn import set_style
set_style("whitegrid")

from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import requests
import os
import re

## Getting the Hot 100 song list
We start by creating a dataframe that contains the features _Title_, _Artists_, _Year_ and _Rank_ first. To do this, we loop through, for each year from 2010 to 2020, and createa  list of all track related features and append to construct one dataframe. Each year $i$'s HTML can be accessed via the webpage `'https://www.billboard.com/charts/year-end/' + str(i) + '/hot-100-songs'`. The features can all be accesed in the following elements
- Title: `<div class="ye-chart-item__title"></div>`
- Artists: `<div class="ye-chart-item__artist"></div>`
- Rank: `<div class="ye-chart-item__rank"></div>`

So we use the `find_all` method to construct a list of all the features

In [2]:
# Constructing an empty list for all features
titles = []
artists = []
ranks = []
years = []

# Constructing the loop
for i in np.arange(2010, 2021, 1):
    # Getting the HTML for each year
    url = 'https://www.billboard.com/charts/year-end/' + str(i) + '/hot-100-songs'
    header = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}
    reqi = Request(url, headers = header)
    htmli = urlopen(reqi)
    soupi = BeautifulSoup(htmli, 'html.parser')
    
    # Constructing lists for year i
    titlei = [str(t.text.replace("\n","")) for t in soupi.find_all('div', {'class':'ye-chart-item__title'})]
    artisti = [a.text.replace("\n","") for a in soupi.find_all('div', {'class':'ye-chart-item__artist'})]
    ranki = [int(r.text.replace("\n","")) for r in soupi.find_all('div', {'class':'ye-chart-item__rank'})]
    yeari = [i for a in range(len(ranki))]
    
    print("For the year", i, "Lengths are", len(titlei), len(artisti), len(ranki), len(yeari))
    # Appending to master list
    titles.extend(titlei)
    artists.extend(artisti)
    ranks.extend(ranki)
    years.extend(yeari)

For the year 2010 Lengths are 100 100 100 100
For the year 2011 Lengths are 99 99 99 99
For the year 2012 Lengths are 100 100 100 100
For the year 2013 Lengths are 100 100 100 100
For the year 2014 Lengths are 100 100 100 100
For the year 2015 Lengths are 100 100 100 100
For the year 2016 Lengths are 99 99 99 99
For the year 2017 Lengths are 100 100 100 100
For the year 2018 Lengths are 100 100 100 100
For the year 2019 Lengths are 100 100 100 100
For the year 2020 Lengths are 100 100 100 100


We now turn the above lists into a dataframe

In [3]:
hot100 = pd.DataFrame({'title':titles, 'artists':artists, 'rank':ranks, 'year':years})
hot100.head(1100)

,title,artists,rank,year
0,TiK ToK,Ke$ha,1,2010
1,Need You Now,Lady Antebellum,2,2010
2,"Hey, Soul Sister",Train,3,2010
3,California Gurls,Katy Perry Featuring Snoop Dogg,4,2010
4,OMG,Usher Featuring will.i.am,5,2010
...,...,...,...,...
1093,More Than My Hometown,Morgan Wallen,96,2020
1094,Lovin' On You,Luke Combs,97,2020
1095,Said Sum,Moneybagg Yo,98,2020
1096,Slide,H.E.R. Featuring YG,99,2020


We can save this dataset to a csv file

In [4]:
hot100.to_csv(r'hot_100.csv', index=False)

### Formatting Title and Artist Names
Since we plan on using the information on artist names and title names to scrape lyrics off of Genius, we need to appropriately format them to make sure the genius URLs work. Genius has a very straightforward way of being able to get to a particular song's lyrics. The url is usually
`https://genius.com/ + artist_name + '-' + title_name + '-lyrics`, where any spaces are replaced by '-'. So we need to ensure that we do the following transformations:
- We first need to make sure that we only use the first/main artist's name
- We need to ensure that there are no special characters (@,\$,etc)

So let's start with formatting!

#### Artist Names
We first need to separate out the first artist's name. Browsing the data, we can see that artists are usually separated based on the following key terms:
- 'Featuring'
- 'With'
- '&'
- '+'

Once separated, we then need to take care of all punctuation and special characters. The code block below first creates a list of artist names and then formats them accordingly to create a new feature `first_artist`.

In [5]:
art_name = hot100['artists'].copy().tolist()

first = []
for art in art_name:
    art = art.rsplit(sep=' Featuring')[0].strip()
    art = art.rsplit(sep=' With')[0].strip()
    art = art.rsplit(sep=',')[0].strip()
    # art = art.rsplit(sep=' &')[0].strip()
    art = art.replace('&', 'and')
    art = art.replace('+', 'and')
    art = art.replace('$','s')
    art = art.replace('P!nk','Pink')
    art = art.replace('!',' ')
    art = art.replace('*',' ')
    art = art.replace('.','')
    art = art.replace('-',' ')
    art = art.replace('  ',' ')
    art = art.lower()
    first.append(art)

type(first)

hot100['first_artist'] = first

hot100.head(1100)

,title,artists,rank,year,first_artist
0,TiK ToK,Ke$ha,1,2010,kesha
1,Need You Now,Lady Antebellum,2,2010,lady antebellum
2,"Hey, Soul Sister",Train,3,2010,train
3,California Gurls,Katy Perry Featuring Snoop Dogg,4,2010,katy perry
4,OMG,Usher Featuring will.i.am,5,2010,usher
...,...,...,...,...,...
1093,More Than My Hometown,Morgan Wallen,96,2020,morgan wallen
1094,Lovin' On You,Luke Combs,97,2020,luke combs
1095,Said Sum,Moneybagg Yo,98,2020,moneybagg yo
1096,Slide,H.E.R. Featuring YG,99,2020,her


#### Title Names
Just like we formatted the artist names to remove special characters, we do the same for song titles. The code block below does this.

In [6]:
## This cell reformats track titles so they generate successful Genius.com urls.
## use the list of procedures a few cells above this one to properly format the titles.

## great an empty list of formatted ('fmtd') track names
title_fmtd = []

hits = hot100['title'].copy().tolist()
for title in hits:
    ## First make the title lower case:
    title = title.lower()
    ## remove , ‘ ( ) ? ! . 
    title = title.replace(',','')
    title = title.replace('\'','')
    title = title.replace('(','')
    title = title.replace(')','')
    title = title.replace('?','')
    title = title.replace('!','')
    title = title.replace('.','')
    ## replace ‘ & ‘ with ‘-’
    title = title.replace(' & ', ' and ')
    ## replace ‘&’ with ‘-‘
    title = title.replace('&', '-')
    
    title_fmtd.append(title)

hot100['title_fmtd'] = title_fmtd
hot100.head(1100)

,title,artists,rank,year,first_artist,title_fmtd
0,TiK ToK,Ke$ha,1,2010,kesha,tik tok
1,Need You Now,Lady Antebellum,2,2010,lady antebellum,need you now
2,"Hey, Soul Sister",Train,3,2010,train,hey soul sister
3,California Gurls,Katy Perry Featuring Snoop Dogg,4,2010,katy perry,california gurls
4,OMG,Usher Featuring will.i.am,5,2010,usher,omg
...,...,...,...,...,...,...
1093,More Than My Hometown,Morgan Wallen,96,2020,morgan wallen,more than my hometown
1094,Lovin' On You,Luke Combs,97,2020,luke combs,lovin on you
1095,Said Sum,Moneybagg Yo,98,2020,moneybagg yo,said sum
1096,Slide,H.E.R. Featuring YG,99,2020,her,slide


We can now save the formatted data to a new dataset.

In [7]:
hot100.to_csv('hot_100_fmtd.csv', index=False)

## Scraping Lyrics off of Genius
Now that we have the artist and title names formatted nicely, we can start with webscraping lyrics for all the songs in our dataset. The following function below has been developed to return a string of lyrics for a song that has been scraped from Genius.com using the song's main artist and the title name. The function has been developed using elements from the following blog posts:
- [How to Leverage Spotify API + Genius Lyrics for Data Science Tasks in Python](https://medium.com/swlh/how-to-leverage-spotify-api-genius-lyrics-for-data-science-tasks-in-python-c36cdfb55cf3)
- [How to Scrape Song Lyrics: A Gentle Tutorial](https://medium.com/analytics-vidhya/how-to-scrape-song-lyrics-a-gentle-python-tutorial-5b1d4ab351d2)

The function is given below, along with an explanation.

In [8]:
def get_lyrics(title, artist):
    if " " in title:
        title_name = str(title.replace(' ','-'))
    else:
        title_name = str(title)
        
    if " " in artist:
        artist_name = str(artist.replace(' ','-'))
    else:
        artist_name = str(artist)
    
    # Loading the URL and parsing it through BeautifulSoup
    url = 'https://genius.com/' + artist_name + '-' + title_name + '-lyrics'
    url = url.replace('--','-')
    header = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}
    req = Request(url, headers = header)
    html = urlopen(req)
    soup = BeautifulSoup(html, 'html.parser')
    
    # Constructing two lyrics_div elements
    lyrics_div1 = soup.find('div', class_='song_body-lyrics')
        
    lyrics_div2 = soup.find('div', {'class':'SongPageGrid-sc-1vi6xda-0 DGVcp Lyrics__Root-sc-1ynbvzw-0 kkHBOZ'})
    
    if lyrics_div1:
        lyrics = lyrics_div1.get_text()
        # Removing stuff in [] (verses, chorus etc)
        lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
        # Removing the song name
        lyrics = lyrics.split('Lyrics', 1)[1]
        # Removing the More on Genius Tag
        lyrics = lyrics.split('More on Genius', 1)[0]
        # Removing empty lines
        lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])
        # Removing new lines and replacing with ;
        lyrics = lyrics.replace('\r\n',' @ ')
    elif lyrics_div2:
        lyrics = lyrics_div2.get_text('\n')
        # Removing stuff in [] (verses, chorus etc)
        lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
        # Removing empty lines
        lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])
        # Removing new lines and replacing with ;
        lyrics = lyrics.replace('\r\n',' @ ')
    elif lyrics_div1 == lyrics_div2 == None:
        lyrics = 'None Found'
    
    return lyrics

What does this function do? Well there are a few parts:
- The function takes two arguments: `title` is the song's title name and `artist` is the name of the (first) artist that performed the song.
- The function first modifies the inputs to remove all whitespaces and replace them with "-"s. **Note that inputs should only contain alphanumeric characters (no ',! etc).**
- Then the function creates the Genius URL for the given song and parses it through BeautifulSoup
- We then create 2 `div` elements that can potentially contain the lyrics. Based on my experience with Genius, the website can generate either of the two elements:
    - `<div class='song_body-lyrics'>`
    - `<div class='SongPageGrid-sc-1vi6xda-0 DGVcp Lyrics__Root-sc-1ynbvzw-0 kkHBOZ'>`
- The function then checks which of the two elements exists. Finding the one that does, it then collects the text for the lyrics and formats it accordingly to generate one clean string. New lines are indicated with the '@' logo.
- The function then returns the formatted lyrics. If no lyrics exist then it returns a string saying 'None Found'

To try it out, let's check what happens when we use the function to get the lyrics for Blinding Lights by The Weeknd!

In [9]:
print(get_lyrics('blinding lights', 'the weeknd'))

Yeah @ I've been tryna call @ I've been on my own for long enough @ Maybe you can show me how to love, maybe @ I'm going through withdrawals @ You don't even have to do too much @ You can turn me on with just a touch, baby @ I look around and @ Sin City's cold and empty  @ No one's around to judge me  @ I can't see clearly when you're gone @ I said, ooh, I'm blinded by the lights @ No, I can't sleep until I feel your touch @ I said, ooh, I'm drowning in the night @ Oh, when I'm like this, you're the one I trust @ Hey, hey, hey @ I'm running out of time @ 'Cause I can see the sun light up the sky @ So I hit the road in overdrive, baby, oh @ The city's cold and empty  @ No one's around to judge me  @ I can't see clearly when you're gone @ I said, ooh, I'm blinded by the lights @ No, I can't sleep until I feel your touch @ I said, ooh, I'm drowning in the night @ Oh, when I'm like this, you're the one I trust @ I'm just calling back to let you know  @ I could never say it on the phone  @ 

### Putting it all together
Now that we have a working function for scraping lyrics, we can, theoretically, iterate over our dataset to append lyrics to each song. However, for some instances in which we are unable to find open the Genius webpage, we can have the loop pass on a string that indicates so.

In [10]:
hot100['lyrics'] = ''

for i in range(len(hot100)):
    print("Song:", i, "Title:", hot100.loc[i,'title_fmtd'], "Artist:", hot100.loc[i, 'first_artist'])
    try:
        hot100.loc[i, 'lyrics'] = get_lyrics(hot100.loc[i,'title_fmtd'], hot100.loc[i, 'first_artist'])
    except:
        hot100.loc[i, 'lyrics'] = 'URL-ERROR-LYRICS-NOT-FOUND'

Song: 0 Title: tik tok Artist: kesha
Song: 1 Title: need you now Artist: lady antebellum
Song: 2 Title: hey soul sister Artist: train
Song: 3 Title: california gurls Artist: katy perry
Song: 4 Title: omg Artist: usher
Song: 5 Title: airplanes Artist: bob
Song: 6 Title: love the way you lie Artist: eminem
Song: 7 Title: bad romance Artist: lady gaga
Song: 8 Title: dynamite Artist: taio cruz
Song: 9 Title: break your heart Artist: taio cruz
Song: 10 Title: nothin on you Artist: bob
Song: 11 Title: i like it Artist: enrique iglesias
Song: 12 Title: bedrock Artist: young money
Song: 13 Title: in my head Artist: jason derulo
Song: 14 Title: rude boy  Artist: rihanna
Song: 15 Title: telephone Artist: lady gaga
Song: 16 Title: teenage dream Artist: katy perry
Song: 17 Title: just the way you are Artist: bruno mars
Song: 18 Title: cooler than me Artist: mike posner
Song: 19 Title: imma be Artist: the black eyed peas
Song: 20 Title: empire state of mind Artist: jay z and alicia keys
Song: 21 Ti

Song: 165 Title: honey bee Artist: blake shelton
Song: 166 Title: dont you wanna stay Artist: jason aldean
Song: 167 Title: we found love Artist: rihanna
Song: 168 Title: pretty girl rock Artist: keri hilson
Song: 169 Title: you and i Artist: lady gaga
Song: 170 Title: like a g6 Artist: far east movement
Song: 171 Title: without you Artist: david guetta
Song: 172 Title: back to december Artist: taylor swift
Song: 173 Title: teenage dream Artist: katy perry
Song: 174 Title: crazy girl Artist: eli young band
Song: 175 Title: cheers drink to that Artist: rihanna
Song: 176 Title: who says Artist: selena gomez and the scene
Song: 177 Title: barefoot blue jean night Artist: jake owen
Song: 178 Title: knee deep Artist: zac brown band
Song: 179 Title: country girl shake it for me Artist: luke bryan
Song: 180 Title: remind me Artist: brad paisley duet
Song: 181 Title: in the dark Artist: dev
Song: 182 Title: backseat Artist: new boyz
Song: 183 Title: headlines Artist: drake
Song: 184 Title: bes

Song: 326 Title: i love it Artist: icona pop
Song: 327 Title: safe and sound Artist: capital cities
Song: 328 Title: treasure Artist: bruno mars
Song: 329 Title: the way Artist: ariana grande
Song: 330 Title: started from the bottom Artist: drake
Song: 331 Title: come and get it Artist: selena gomez
Song: 332 Title: hold on were going home Artist: drake
Song: 333 Title: daylight Artist: maroon 5
Song: 334 Title: feel this moment Artist: pitbull
Song: 335 Title: applause Artist: lady gaga
Song: 336 Title: one more night Artist: maroon 5
Song: 337 Title: love me Artist: lil wayne
Song: 338 Title: my songs know what you did in the dark light em up Artist: fall out boy
Song: 339 Title: f**kin problems Artist: asap rocky
Song: 340 Title: beauty and a beat Artist: justin bieber
Song: 341 Title: same love Artist: macklemore and ryan lewis
Song: 342 Title: sweet nothing Artist: calvin harris
Song: 343 Title: summertime sadness Artist: lana del rey and cedric gervais
Song: 344 Title: home Artis

Song: 492 Title: stay the night Artist: zedd
Song: 493 Title: partition Artist: beyonce
Song: 494 Title: studio Artist: schoolboy q
Song: 495 Title: 0 to 100 / the catch up Artist: drake
Song: 496 Title: i dont dance Artist: lee brice
Song: 497 Title: somethin bad Artist: miranda lambert duet
Song: 498 Title: adore you Artist: miley cyrus
Song: 499 Title: uptown funk Artist: mark ronson
Song: 500 Title: thinking out loud Artist: ed sheeran
Song: 501 Title: see you again Artist: wiz khalifa
Song: 502 Title: trap queen Artist: fetty wap
Song: 503 Title: sugar Artist: maroon 5
Song: 504 Title: shut up and dance Artist: walk the moon
Song: 505 Title: blank space Artist: taylor swift
Song: 506 Title: watch me Artist: silento
Song: 507 Title: earned it fifty shades of grey Artist: the weeknd
Song: 508 Title: the hills Artist: the weeknd
Song: 509 Title: cheerleader Artist: omi
Song: 510 Title: cant feel my face Artist: the weeknd
Song: 511 Title: love me like you do Artist: ellie goulding
So

Song: 657 Title: for free Artist: dj khaled
Song: 658 Title: never be like you Artist: flume
Song: 659 Title: in the night Artist: the weeknd
Song: 660 Title: me too Artist: meghan trainor
Song: 661 Title: exs and ohs Artist: elle king
Song: 662 Title: die a happy man Artist: thomas rhett
Song: 663 Title: white iverson Artist: post malone
Song: 664 Title: close Artist: nick jonas
Song: 665 Title: unsteady Artist: x ambassadors
Song: 666 Title: sucker for pain Artist: lil wayne
Song: 667 Title: down in the dm Artist: yo gotti
Song: 668 Title: luv Artist: tory lanez
Song: 669 Title: sorry Artist: beyonce
Song: 670 Title: cant feel my face Artist: the weeknd
Song: 671 Title: hymn for the weekend Artist: coldplay
Song: 672 Title: say it Artist: tory lanez
Song: 673 Title: antidote Artist: travis scott
Song: 674 Title: lost boy Artist: ruth b
Song: 675 Title: side to side Artist: ariana grande
Song: 676 Title: sit still look pretty Artist: daya
Song: 677 Title: wildest dreams Artist: taylor

Song: 822 Title: yes indeed Artist: lil baby and drake
Song: 823 Title: friends Artist: marshmello and anne marie
Song: 824 Title: bad at love Artist: halsey
Song: 825 Title: taste Artist: tyga
Song: 826 Title: let you down Artist: nf
Song: 827 Title: no limit Artist: g eazy
Song: 828 Title: fefe Artist: 6ix9ine
Song: 829 Title: tequila Artist: dan and shay
Song: 830 Title: feel it still Artist: portugal the man
Song: 831 Title: motorsport Artist: migos
Song: 832 Title: i like me better Artist: lauv
Song: 833 Title: youngblood Artist: 5 seconds of summer
Song: 834 Title: whatever it takes Artist: imagine dragons
Song: 835 Title: ric flair drip Artist: offset and metro boomin
Song: 836 Title: i fall apart Artist: post malone
Song: 837 Title: pray for me Artist: the weeknd and kendrick lamar
Song: 838 Title: back to you Artist: selena gomez
Song: 839 Title: sicko mode Artist: travis scott
Song: 840 Title: walk it talk it Artist: migos
Song: 841 Title: gucci gang Artist: lil pump
Song: 84

Song: 986 Title: one thing right Artist: marshmello and kane brown
Song: 987 Title: cash shit Artist: megan thee stallion
Song: 988 Title: tequila Artist: dan and shay
Song: 989 Title: shotta flow Artist: nle choppa
Song: 990 Title: hot girl summer Artist: megan thee stallion
Song: 991 Title: talk you out of it Artist: florida georgia line
Song: 992 Title: beautiful Artist: bazzi
Song: 993 Title: eyes on you Artist: chase rice
Song: 994 Title: all to myself Artist: dan and shay
Song: 995 Title: boyfriend Artist: ariana grande and social house
Song: 996 Title: walk me home Artist: pink
Song: 997 Title: robbery Artist: juice wrld
Song: 998 Title: blinding lights Artist: the weeknd
Song: 999 Title: circles Artist: post malone
Song: 1000 Title: the box Artist: roddy ricch
Song: 1001 Title: dont start now Artist: dua lipa
Song: 1002 Title: rockstar Artist: dababy
Song: 1003 Title: adore you Artist: harry styles
Song: 1004 Title: life is good Artist: future
Song: 1005 Title: memories Artist:

In [11]:
hot100.head(1100)

,title,artists,rank,year,first_artist,title_fmtd,lyrics
0,TiK ToK,Ke$ha,1,2010,kesha,tik tok,Wake up in the morning feelin' like P. Diddy @...
1,Need You Now,Lady Antebellum,2,2010,lady antebellum,need you now,[Intro: Hillary Scott & @ Charles Kelley @ ] ...
2,"Hey, Soul Sister",Train,3,2010,train,hey soul sister,Heyy @ He-e-e-e-ey @ He-e-e-e-ey @ Your lipsti...
3,California Gurls,Katy Perry Featuring Snoop Dogg,4,2010,katy perry,california gurls,"Greetings, loved ones @ Let's take a journey @..."
4,OMG,Usher Featuring will.i.am,5,2010,usher,omg,[Intro: will.i.am & @ Usher @ ] @ Oh my gosh ...
...,...,...,...,...,...,...,...
1093,More Than My Hometown,Morgan Wallen,96,2020,morgan wallen,more than my hometown,"Girl, our mamas are best friends and so are we..."
1094,Lovin' On You,Luke Combs,97,2020,luke combs,lovin on you,Don't get me wrong @ I like a bobber on the wa...
1095,Said Sum,Moneybagg Yo,98,2020,moneybagg yo,said sum,"( @ Turn me up, YC @ ) @ Huh? @ Ah, I thought..."
1096,Slide,H.E.R. Featuring YG,99,2020,her,slide,You always wearin' them glasses @ You don't wa...


Now let's save this to a csv file!

In [12]:
hot100.to_csv('hot_100_lyrics.csv', index=False)

What about the data that doesn't contain any lyrics? Well we can filter it out by using the following keywords for lyrics:
- `hot100['lyrics'] == 'URL-ERROR-LYRICS-NOT-FOUND'`
- `hot100['lyrics'] == 'None Found'`

In [13]:
hot100 = pd.read_csv('hot_100_lyrics.csv')

In [14]:
hot100_nolyrics = hot100[(hot100['lyrics'] == 'URL-ERROR-LYRICS-NOT-FOUND') | (hot100['lyrics'] == 'None Found')].copy()
hot100_nolyrics.to_csv('hot_100_nolyrics.csv', index=False)
hot100_nolyrics

,title,artists,rank,year,first_artist,title_fmtd,lyrics
20,Empire State Of Mind,Jay-Z + Alicia Keys,21,2010,jay z and alicia keys,empire state of mind,URL-ERROR-LYRICS-NOT-FOUND
49,Young Forever,Jay-Z + Mr. Hudson,50,2010,jay z and mr hudson,young forever,URL-ERROR-LYRICS-NOT-FOUND
117,F**kin' Perfect,P!nk,19,2011,pink,f**kin perfect,URL-ERROR-LYRICS-NOT-FOUND
152,Stereo Love,Edward Maya & Vika Jigulina,54,2011,edward maya and vika jigulina,stereo love,URL-ERROR-LYRICS-NOT-FOUND
180,Remind Me,Brad Paisley Duet With Carrie Underwood,82,2011,brad paisley duet,remind me,URL-ERROR-LYRICS-NOT-FOUND
238,Ni**as in Paris,Jay Z Kanye West,40,2012,jay z kanye west,ni**as in paris,URL-ERROR-LYRICS-NOT-FOUND
255,Dance (A$$),Big Sean Featuring Nicki Minaj,57,2012,big sean,dance a$$,URL-ERROR-LYRICS-NOT-FOUND
319,Cups (Pitch Perfect's When I'm Gone),Anna Kendrick,21,2013,anna kendrick,cups pitch perfects when im gone,URL-ERROR-LYRICS-NOT-FOUND
321,Scream & Shout,will.i.am & Britney Spears,23,2013,william and britney spears,scream and shout,URL-ERROR-LYRICS-NOT-FOUND
339,F**kin Problems,"A$AP Rocky Featuring Drake, 2 Chainz & Kendric...",41,2013,asap rocky,f**kin problems,URL-ERROR-LYRICS-NOT-FOUND


Written by Mir Adnan Mahmood for the Erdos Institute's DataScience Bootcamp project.